In [1]:
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools
from perses.tests.test_topology_proposal import generate_dipeptide_top_pos_sys, generate_atp
import pickle
import copy
from perses.annihilation.relative import RestCapablePMEHybridTopologyFactory

#############################################
# CONSTANTS
#############################################
temperature = 298.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CUDA")


conducting subsequent work with the following platform: CPU


INFO:rdkit:Enabling RDKit 2021.03.4 jupyter extensions


In [2]:
def test_bond_energies(htf, is_old=True, is_solvated=False, check_scale=False):
    htf_copy = copy.deepcopy(htf)
    
    # Get harmonic bond force and old/new positions
    system = htf._topology_proposal.old_system if is_old else htf._topology_proposal.new_system
    harmonic_bond_force = system.getForce(0) 
    positions = htf.old_positions(htf.hybrid_positions) if is_old else htf.new_positions(htf.hybrid_positions)
    
    # Get custom bond force and hybrid positions
    bond_force_index = 1 if is_solvated else 0
    hybrid_system = htf.hybrid_system
    custom_bond_force = hybrid_system.getForce(bond_force_index)
    hybrid_positions = htf.hybrid_positions
    
    # Remove all other forces
    for i in range(hybrid_system.getNumForces(), hybrid_system.getNumForces() - 4, -1):
        hybrid_system.removeForce(i - 1)
    
    # Set global parameters
    lambda_old = 1 if is_old else 0
    lambda_new = 0 if is_old else 1
    for i in range(custom_bond_force.getNumGlobalParameters()):
        if custom_bond_force.getGlobalParameterName(i) == 'lambda_alchemical_bonds_old':
            custom_bond_force.setGlobalParameterDefaultValue(i, lambda_old)
        if custom_bond_force.getGlobalParameterName(i) == 'lambda_alchemical_bonds_new':
            custom_bond_force.setGlobalParameterDefaultValue(i, lambda_new)

#     # Zero the unique old/new bonds in the custom bond force
#     hybrid_to_bond_indices = htf._hybrid_to_new_bond_indices if is_old else htf._hybrid_to_old_bond_indices
#     for hybrid_idx, idx in hybrid_to_bond_indices.items():
#         p1, p2, hybrid_params = custom_bond_force.getBondParameters(hybrid_idx)
#         hybrid_params = list(hybrid_params)
#         index_to_zero = -3 if is_old else -1
#         hybrid_params[index_to_zero] *= 0
#         custom_bond_force.setBondParameters(hybrid_idx, p1, p2, hybrid_params)

    ## Get energy components of standard bond force
    platform = configure_platform(REFERENCE_PLATFORM)
    thermostate_other = ThermodynamicState(system=system, temperature=temperature)
    integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
    context_other = thermostate_other.create_context(integrator_other)
    context_other.setPositions(positions)
    components_other = compute_potential_components(context_other, beta=beta)
    print(components_other)
    
    ## Get energy components of custom bond force
    if is_solvated:
        custom_bond_force.setUsesPeriodicBoundaryConditions(True)
    thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
    integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
    context_hybrid.setPositions(hybrid_positions)
    components_hybrid = compute_potential_components(context_hybrid, beta=beta)
    print(components_hybrid)
    
    assert np.isclose([components_other[0][1]], [components_hybrid[0][1]])
    
    print("Success! Custom bond force and standard bond force energies are equal!")
    
    if check_scale:
        
        # Get custom bond force and hybrid positions
        bond_force_index = 1 if is_solvated else 0
        hybrid_system = htf_copy.hybrid_system
        custom_bond_force = hybrid_system.getForce(bond_force_index)
        hybrid_positions = htf_copy.hybrid_positions
        
        if is_solvated:
            custom_bond_force.setUsesPeriodicBoundaryConditions(True)
        
        # Remove all other forces
        for i in range(hybrid_system.getNumForces(), hybrid_system.getNumForces() - 4, -1):
            hybrid_system.removeForce(i - 1)
        
        ## Get energy components of custom bond force
        thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
        integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
        context_hybrid.setPositions(hybrid_positions)
        components_hybrid = compute_potential_components(context_hybrid, beta=beta)
        print(components_hybrid)
        
        # Set `scale_lambda_{i}` to 0.5
        for i in range(custom_bond_force.getNumGlobalParameters()):
            if custom_bond_force.getGlobalParameterName(i) == 'lambda_rest_bonds':
                custom_bond_force.setGlobalParameterDefaultValue(i, 0.5)
        
        ## Get energy components of custom bond force with scaling
        thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
        integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
        context_hybrid.setPositions(hybrid_positions)
        components_hybrid_scaled = compute_potential_components(context_hybrid, beta=beta)
        print(components_hybrid_scaled)
        
        assert not np.isclose([components_hybrid[0][1]], [components_hybrid_scaled[0][1]])
        
        print("Success! Scaling the bond force changes the energy")

# Alanine dipeptide in vacuum

In [35]:
# Create a htf
atp, system_generator = generate_atp(phase = 'vacuum')
topology_proposal, new_positions, _, _ = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG2, atomic number=6), Atom(name=OG1, atomic number=8), Atom(name=HB, atomic number=1), Atom(name=HG1, atomic number=1), Atom(name=HG21, atomic number=1), Atom(name=HG22, atomic number=1), Atom(name=HG23, atomic number=1)]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 6
INFO:geometry:Atom index proposal order is [18, 14, 19, 17, 15, 16]
INFO:geometry:omitted_bonds: []
INFO:geometry:d

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions
INFO:geometry:There are 6 new atoms
INFO:geometry:	reduced angle potential = 0.1916210312398112.


conducting subsequent work with the following platform: CUDA


INFO:geometry:	reduced angle potential = 0.5294767290459792.
INFO:geometry:	reduced angle potential = 1.4198399455193607.
INFO:geometry:	reduced angle potential = 0.10363537635946199.
INFO:geometry:	reduced angle potential = 0.09980266220072555.
INFO:geometry:	reduced angle potential = 0.20610506455409472.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 angle forces in the no-nonbonded final system
INFO:geometry:	there are 72 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


...

INFO:geometry:	reduced angle potential = 1.2915588460963948e-10.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 9 bond forces in the no-nonbonded final system
INFO:geometry:	there are 36 angle forces in the no-nonbonded final system
INFO:geometry:	there are 42 torsion forces in the no-nonbonded final system
INFO:geometry:reverse final system defined with 0 neglected angles.


conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 16.815513850992346


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 0.00017810081214392654), ('CustomTorsionForce', 0.002891174165036068), ('CustomBondForce', 4.777690951809965)]


INFO:geometry:total reduced energy added from growth system: 4.780760226787146
INFO:geometry:final reduced energy 21.59627388135083
INFO:geometry:sum of energies: 21.59627407777949
INFO:geometry:magnitude of difference in the energies: 1.9642866355695787e-07
INFO:geometry:Final logp_proposal: -17727.595083382985


In [4]:
flatten_exceptions = True
flatten_torsions = True
endstate = None
rest_region = None
htf = RestCapablePMEHybridTopologyFactory(topology_proposal=topology_proposal,
                     current_positions=atp.positions,
                     new_positions=new_positions,
                     rest_region=rest_region,
                     use_dispersion_correction=False,
                     generate_htf_for_testing=True,
                     interpolate_old_and_new_14s=flatten_exceptions
                    # no flatten torsions yet
                    )

INFO:relative:*** Generating RestCapablePMEHybridTopologyFactory ***
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:relative:Generating old system exceptions dict...
INFO:relative:Generating new system exceptions dict...
INFO:relative:

In [5]:
htf._r_cutoff

Quantity(value=100, unit=nanometer)

In [6]:
with open("atp_vacuum.pickle", "wb") as f:
    pickle.dump(htf, f)

In [7]:
with open("atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [8]:
htf.hybrid_system.getForces()

[<openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2aadb46ba900> >,
 <openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2aadb46ba9c0> >,
 <openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2aadb46ba7b0> >,
 <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2aadb46ba870> >,
 <openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2aadb46ba6c0> >,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2aadb46ba750> >]

In [9]:
htf.hybrid_system.getForce(5).getIncludeDirectSpace()

False

In [7]:
from simtk.openmm import VerletIntegrator

In [12]:
htf._topology_proposal.old_system.removeForce(2)
htf._topology_proposal.old_system.removeForce(1)
htf._topology_proposal.old_system.removeForce(0)

In [14]:
# Check the potential energy of the nb force (total energy)
integrator = VerletIntegrator(0.001)
context = openmm.Context(htf._topology_proposal.old_system, integrator)
context.setPositions(htf.old_positions(htf.hybrid_positions))
context.getState(getEnergy=True).getPotentialEnergy()

Quantity(value=-97.74547495983268, unit=kilojoule/mole)

In [10]:
htf._topology_proposal.new_system.removeForce(2)
htf._topology_proposal.new_system.removeForce(1)
htf._topology_proposal.new_system.removeForce(0)

In [11]:
# Check the potential energy of the nb force (total energy)
integrator = VerletIntegrator(0.001)
context = openmm.Context(htf._topology_proposal.new_system, integrator)
context.setPositions(htf.new_positions(htf.hybrid_positions))
context.getState(getEnergy=True).getPotentialEnergy()

Quantity(value=1441.296769774057, unit=kilojoule/mole)

### Test one alchemical region, no rest region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [28]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [11]:
test_bond_energies(htf)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.0347836535652138), ('HarmonicAngleForce', 0.6112775462128993), ('PeriodicTorsionForce', 16.285398289983057), ('NonbondedForce', -39.449927745600284), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.0347836535652138), ('CustomAngleForce', 0.6112775462128993), ('AndersenThermostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [12]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [13]:
test_bond_energies(htf, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 2.0790338903939647), ('HarmonicAngleForce', 3.430235820649689), ('PeriodicTorsionForce', 26.404036436796364), ('NonbondedForce', 1124.682158104184), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 2.0790338903939647), ('CustomAngleForce', 0.6112775462128993), ('AndersenThermostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


### Test one alchemical region, one rest region

In [69]:
# Create a htf
atp, system_generator = generate_atp(phase = 'vacuum')
topology_proposal, new_positions, _, _ = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG2, atomic number=6), Atom(name=OG1, atomic number=8), Atom(name=HB, atomic number=1), Atom(name=HG1, atomic number=1), Atom(name=HG21, atomic number=1), Atom(name=HG22, atomic number=1), Atom(name=HG23, atomic number=1)]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 6
INFO:geometry:Atom index proposal order is [18, 14, 16, 17, 15, 19]
INFO:geometry:omitted_bonds: []
INFO:geometry:d

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions
INFO:geometry:There are 6 new atoms
INFO:geometry:	reduced angle potential = 0.002462794086282191.


conducting subsequent work with the following platform: CUDA


INFO:geometry:	reduced angle potential = 2.368313509650312.
INFO:geometry:	reduced angle potential = 0.11423804670668047.
INFO:geometry:	reduced angle potential = 0.0412920616254721.
INFO:geometry:	reduced angle potential = 1.459823375546265.
INFO:geometry:	reduced angle potential = 0.26981558425144714.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 angle forces in the no-nonbonded final system
INFO:geometry:	there are 72 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


...

INFO:geometry:	reduced angle potential = 7.39096069988752e-11.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 9 bond forces in the no-nonbonded final system
INFO:geometry:	there are 36 angle forces in the no-nonbonded final system
INFO:geometry:	there are 42 torsion forces in the no-nonbonded final system
INFO:geometry:reverse final system defined with 0 neglected angles.


conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 16.815513850992346


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 0.00017810081214392654), ('CustomTorsionForce', 0.002891174165036068), ('CustomBondForce', 4.777690951809965)]


INFO:geometry:total reduced energy added from growth system: 4.780760226787146
INFO:geometry:final reduced energy 21.59627388135083
INFO:geometry:sum of energies: 21.59627407777949
INFO:geometry:magnitude of difference in the energies: 1.9642866355695787e-07
INFO:geometry:Final logp_proposal: -17980.99178064207


In [70]:
flatten_exceptions = True
flatten_torsions = True
endstate = None
rest_region = list(htf._atom_classes['unique_old_atoms']) + list(htf._atom_classes['unique_new_atoms']) + [10, 12]
htf = RestCapablePMEHybridTopologyFactory(topology_proposal=topology_proposal,
                     current_positions=atp.positions,
                     new_positions=new_positions,
                     rest_region=rest_region,
                     use_dispersion_correction=False,
                     generate_htf_for_testing=True,
                     interpolate_old_and_new_14s=flatten_exceptions
                    )


INFO:relative:*** Generating RestCapablePMEHybridTopologyFactory ***
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:relative:Generating old system exceptions dict...
INFO:relative:Generating new system exceptions dict...
INFO:relative:

In [71]:
with open("atp_vacuum_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [72]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [73]:
test_bond_energies(htf, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.0347836535652138), ('HarmonicAngleForce', 0.6112775462128993), ('PeriodicTorsionForce', 16.285398289983057), ('NonbondedForce', -39.449927745600284), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.0347836535652138), ('CustomAngleForce', 0.6112775462128993), ('AndersenThermostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.0347836535652138), ('CustomAngleForce', 0.6112775462128993), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.03452180595078771), ('CustomAngleForce', 0.6112775462128993), ('AndersenThermostat', 0.0)]
Success! Scaling the bond force changes the energy


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [74]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [75]:
test_bond_energies(htf, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.35077936084123806), ('HarmonicAngleForce', 9.272449786711276), ('PeriodicTorsionForce', 26.163000081487677), ('NonbondedForce', 9765.584801666835), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.35077936084123806), ('CustomAngleForce', 0.6112775462128993), ('AndersenThermostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


# Alanine dipeptide in solvent

In [5]:
# Create a htf
atp, system_generator = generate_atp(phase = 'solvent')
topology_proposal, new_positions, _, _ = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG2, atomic number=6), Atom(name=OG1, atomic number=8), Atom(name=HB, atomic number=1), Atom(name=HG1, atomic number=1), Atom(name=HG21, atomic number=1), Atom(name=HG22, atomic number=1), Atom(name=HG23, atomic number=1)]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 6
INFO:geometry:Atom index proposal order is [14, 18, 17, 19, 15, 16]
INFO:geometry:omitted_bonds: []
INFO:geometry:d

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1654 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (19, 18, 10, 8, [1, Quantity(value=-0.07

Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/build_artifacts/openmm_1629263119354/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU


INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 angle forces in the no-nonbonded final system
INFO:geometry:	there are 72 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


...

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/build_artifacts/openmm_1629263119354/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU
added energy components: [('CustomBondForce', 0.5978518668881), ('CustomAngleForce', 5.416667964161844), ('CustomTorsionForce', 10.345145951898028), ('CustomBondForce', -68.22753538705471)]


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 9 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 36 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 42 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1631 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (6, 8, 10, 12, [1, Quantity(value=-2.0823

Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/build_artifacts/openmm_1629263119354/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU


INFO:geometry:total reduced potential before atom placement: 16.815513985240173
INFO:geometry:total reduced energy added from growth system: 5.747545172611741
INFO:geometry:final reduced energy 22.563059220440383
INFO:geometry:sum of energies: 22.563059157851914
INFO:geometry:magnitude of difference in the energies: 6.258846951823216e-08
INFO:geometry:Final logp_proposal: -17765.61580635068


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/build_artifacts/openmm_1629263119354/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU
added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 0.00017809240723536907), ('CustomTorsionForce', 0.0028911445554170986), ('CustomBondForce', 5.744475935649088)]


In [6]:
flatten_exceptions = True
flatten_torsions = True
endstate = None
rest_region = None
htf = RestCapablePMEHybridTopologyFactory(topology_proposal=topology_proposal,
                     current_positions=atp.positions,
                     new_positions=new_positions,
                     rest_region=rest_region,
                     use_dispersion_correction=False,
                     generate_htf_for_testing=True,
                     interpolate_old_and_new_14s=flatten_exceptions
                    )

INFO:relative:*** Generating RestCapablePMEHybridTopologyFactory ***
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.56477354, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.56477354, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.56477354), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:relative:Generating old system exceptions dict...
INFO:re

In [7]:
with open("atp_solvent.pickle", "wb") as f:
    pickle.dump(htf, f)

### Check that the reciprocal space is working as expected for ala dipeptide in solvent

In [4]:
from simtk.openmm import XmlSerializer


In [3]:
with open("atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [5]:
with open('system.xml', 'w') as outfile:
    system_xml = XmlSerializer.serialize(htf.hybrid_system)
    outfile.write(system_xml)

In [8]:
from simtk.openmm import VerletIntegrator

In [36]:
htf._topology_proposal.old_system.getForces()

[<openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2abab3598b40> >,
 <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2abab3598ae0> >,
 <openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2abab3598e70> >,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2abab3598300> >,
 <openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2abab3598e10> >]

In [37]:
# Delete all forces except the nb force
htf._topology_proposal.old_system.removeForce(4)
htf._topology_proposal.old_system.removeForce(2)
htf._topology_proposal.old_system.removeForce(1)
htf._topology_proposal.old_system.removeForce(0)

In [38]:
htf._topology_proposal.old_system.getForces()

[<openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2abab36303f0> >]

In [39]:
# Set the reciprocal space force group to 1
htf._topology_proposal.old_system.getForce(0).setReciprocalSpaceForceGroup(1)

In [40]:
# Check that the direct space force group is 0
htf._topology_proposal.old_system.getForce(0).getForceGroup()

0

In [41]:
# Check that the reciprocal space force group is 1
htf._topology_proposal.old_system.getForce(0).getReciprocalSpaceForceGroup()

1

In [9]:
# Check the potential energy of the nb force (total energy)
integrator = VerletIntegrator(0.001)
context = openmm.Context(htf._topology_proposal.old_system, integrator)
context.setPositions(htf.old_positions(htf.hybrid_positions))
context.getState(getEnergy=True).getPotentialEnergy()

Quantity(value=-2317.135130805541, unit=kilojoule/mole)

In [11]:
# Check the potential energy of the nb force (total energy)
integrator = VerletIntegrator(0.001)
context = openmm.Context(htf._topology_proposal.old_system, integrator)
context.setPositions(htf.old_positions(htf.hybrid_positions))
state = context.getState(getPositions=True)

In [12]:
with open('state.xml', 'w') as outfile:
    state_xml = XmlSerializer.serialize(state)
    outfile.write(state_xml)

In [13]:
with open("topology.pickle", "wb") as f:
    pickle.dump(htf.hybrid_topology, f)

In [43]:
# Check the potential energy of the direct space of the nb force 
integrator = VerletIntegrator(0.001)
context = openmm.Context(htf._topology_proposal.old_system, integrator)
context.setPositions(htf.old_positions(htf.hybrid_positions))
context.getState(getEnergy=True, groups=set([0])).getPotentialEnergy()

Quantity(value=119792.39420779605, unit=kilojoule/mole)

In [44]:
# Check the potential energy of the reciprocal space of the nb force 
integrator = VerletIntegrator(0.001)
context = openmm.Context(htf._topology_proposal.old_system, integrator)
context.setPositions(htf.old_positions(htf.hybrid_positions))
context.getState(getEnergy=True, groups=set([1])).getPotentialEnergy()

Quantity(value=-122153.15459813026, unit=kilojoule/mole)

In [45]:
# Check that the direct space + reciprocal space = total energy
119792.39420779605 + -122153.15459

-2360.7603822039528

In [46]:
# Now disable the direct space
htf._topology_proposal.old_system.getForce(0).setIncludeDirectSpace(False)

In [48]:
# Check that the energy of the system with direct space enabled is the same as the energy of force group 1
integrator = VerletIntegrator(0.001)
context = openmm.Context(htf._topology_proposal.old_system, integrator)
context.setPositions(htf.old_positions(htf.hybrid_positions))
context.getState(getEnergy=True).getPotentialEnergy()

Quantity(value=-122153.15459813026, unit=kilojoule/mole)

### Test one alchemical region, no rest regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [53]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [54]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521467), ('HarmonicAngleForce', 0.6112775462129013), ('PeriodicTorsionForce', 16.285398289983075), ('NonbondedForce', -913.4899398562932), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.0347836535652138), ('CustomAngleForce', 0.6112775462128993), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [55]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [56]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.5354578445848086), ('HarmonicAngleForce', 7.390279777724092), ('PeriodicTorsionForce', 26.146835612586532), ('NonbondedForce', -573.4656458412903), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.5354578445848076), ('CustomAngleForce', 0.6112775462128993), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


### Test one alchemical region, one rest region

In [76]:
rest_region = list(htf._atom_classes['unique_old_atoms']) + list(htf._atom_classes['unique_new_atoms']) + [10, 12]

In [77]:
# Create a htf
atp, system_generator = generate_atp(phase = 'solvent')
topology_proposal, new_positions, _, _ = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG2, atomic number=6), Atom(name=OG1, atomic number=8), Atom(name=HB, atomic number=1), Atom(name=HG1, atomic number=1), Atom(name=HG21, atomic number=1), Atom(name=HG22, atomic number=1), Atom(name=HG23, atomic number=1)]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 6


making topology proposal
generating geometry engine
making geometry proposal from ALA to THR


INFO:geometry:Atom index proposal order is [14, 18, 15, 16, 19, 17]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1654 in the reference

conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions
INFO:geometry:There are 6 new atoms
INFO:geometry:	reduced angle potential = 2.438674416988987.


...

INFO:geometry:	reduced angle potential = 7.39096069988752e-11.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 9 bond forces in the no-nonbonded final system
INFO:geometry:	there are 36 angle forces in the no-nonbonded final system
INFO:geometry:	there are 42 torsion forces in the no-nonbonded final system
INFO:geometry:reverse final system defined with 0 neglected angles.


conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 16.815513850992346


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 0.00017810081214392654), ('CustomTorsionForce', 0.002891174165036068), ('CustomBondForce', 4.777690951809965)]


INFO:geometry:total reduced energy added from growth system: 4.780760226787146
INFO:geometry:final reduced energy 21.596273881350857
INFO:geometry:sum of energies: 21.59627407777949
INFO:geometry:magnitude of difference in the energies: 1.9642863513524844e-07
INFO:geometry:Final logp_proposal: -17931.595052951918


In [78]:
flatten_exceptions = True
flatten_torsions = True
endstate = None
rest_region = rest_region
htf = RestCapablePMEHybridTopologyFactory(topology_proposal=topology_proposal,
                     current_positions=atp.positions,
                     new_positions=new_positions,
                     rest_region=rest_region,
                     use_dispersion_correction=False,
                     generate_htf_for_testing=True,
                     interpolate_old_and_new_14s=flatten_exceptions
                    )

INFO:relative:*** Generating RestCapablePMEHybridTopologyFactory ***
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.56477354, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.56477354, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.56477354), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:relative:Generating old system exceptions dict...
INFO:re

In [79]:
with open("atp_solvent_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [80]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [81]:
test_bond_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521467), ('HarmonicAngleForce', 0.6112775462129013), ('PeriodicTorsionForce', 16.285398289983075), ('NonbondedForce', -854.4284339778609), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.0347836535652138), ('CustomAngleForce', 0.6112775462128993), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.0347836535652138), ('CustomAngleForce', 0.6112775462128993), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.03452180595078771), ('CustomAngleForce', 0.6112775462128993), ('AndersenThermostat', 

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [82]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [83]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.6396456066518906), ('HarmonicAngleForce', 242.87667340273873), ('PeriodicTorsionForce', 28.018682089890692), ('NonbondedForce', -823.8106185152285), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.6396456066518919), ('CustomAngleForce', 0.6112775462128993), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


# CDK2 transformation in solvent

In [18]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


In [19]:
# Create a htf
setup_directory = resource_filename("perses", "data/cdk2-example")

# Get options
from perses.app.setup_relative_calculation import getSetupOptions
yaml_filename = os.path.join(setup_directory, "cdk2_setup_repex.yaml")
setup_options = getSetupOptions(yaml_filename)

# Update options
for parameter in ['protein_pdb', 'ligand_file']:
    setup_options[parameter] = os.path.join(setup_directory, setup_options[parameter])
    
setup_options['rest_over_protocol'] = True
setup_options['phases'] = ['solvent']
setup_options['use_given_geometries'] = False
setup_options['rest_region'] = None
setup_options['validate_endstate_energies'] = False
    
setup_dict = setup_relative_calculation.run_setup(setup_options, serialize_systems=False, build_samplers=False)

2021-08-19 13:28:17,274:(0.00s):root:	Detecting phases...
2021-08-19 13:28:17,275:(0.00s):root:		phases detected: ['complex', 'solvent', 'vacuum']
2021-08-19 13:28:17,277:(0.00s):root:No constraints will be removed
2021-08-19 13:28:17,278:(0.00s):root:No spectators
2021-08-19 13:28:17,280:(0.00s):root:			run_type is not specified; default to None
2021-08-19 13:28:17,281:(0.00s):root:	Detecting fe_type...
2021-08-19 13:28:17,282:(0.00s):root:		fe_type: repex
2021-08-19 13:28:17,283:(0.00s):root:			offline-freq not specified: default to 10.
2021-08-19 13:28:17,285:(0.00s):root:	'neglect_angles' detected: False.
2021-08-19 13:28:17,286:(0.00s):root:	'softcore_v2' not specified: default to 'False'
2021-08-19 13:28:17,288:(0.00s):root:	Creating 'output'...
2021-08-19 13:28:17,290:(0.00s):root:	detected ligand file: /home/zhangi/miniconda3/envs/perses-rest-over-protocol/lib/python3.8/site-packages/perses-0.9.1+35.gb0f7541-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf
2021-08-19 13:28:1

2021-08-19 13:28:19,748:(0.00s):geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
2021-08-19 13:28:19,749:(0.00s):geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
2021-08-19 13:28:19,750:(0.00s):geometry:		there are 2395 in the reference Nonbonded force
2021-08-19 13:28:19,790:(0.04s):geometry:Neglected angle terms : []
2021-08-19 13:28:19,792:(0.00s):geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
2021-08-19 13:28:19,793:(0.00s):geometry:extra torsions: {0: (2190, 2161, 2162, 2163, [1, Quantity(value=-0.019594642035228205, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 1]), 1: (2190, 2161, 2162, 2191, [1, Quantity(value=-3.1361896196768977, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 1]), 2: (2190, 2161, 2175, 2174, [1, Quantity(value=-0.008871224163264912, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole)

conducting subsequent work with the following platform: CUDA


2021-08-19 13:28:20,302:(0.51s):geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


...

2021-08-19 13:28:25,928:(1.01s):geometry:total reduced potential before atom placement: 48.41899607369697


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-08-19 13:28:27,090:(1.16s):geometry:total reduced energy added from growth system: -5.038066954807732
2021-08-19 13:28:27,092:(0.00s):geometry:final reduced energy 43.38092980084384
2021-08-19 13:28:27,094:(0.00s):geometry:sum of energies: 43.380929118889235
2021-08-19 13:28:27,095:(0.00s):geometry:magnitude of difference in the energies: 6.819546030101264e-07
2021-08-19 13:28:27,097:(0.00s):geometry:Final logp_proposal: 11.810926560573073
2021-08-19 13:28:27,230:(0.13s):root:	writing pickle output...


added energy components: [('CustomBondForce', 0.08798429542805747), ('CustomAngleForce', 0.04605409749261914), ('CustomTorsionForce', 0.005423562569706602), ('CustomBondForce', -5.177528910298116)]


2021-08-19 13:28:27,397:(0.17s):root:	successfully dumped pickle.
2021-08-19 13:28:27,404:(0.01s):root:	setup is complete.  Writing proposals and positions for each phase to top_prop dict...
2021-08-19 13:28:27,406:(0.00s):root:	writing atom_mapping.png
2021-08-19 13:28:27,457:(0.05s):root:	 steps per move application: 50
2021-08-19 13:28:27,459:(0.00s):root:	trajectory prefix: out
2021-08-19 13:28:27,461:(0.00s):root:	atom selection detected: not water
2021-08-19 13:28:27,463:(0.00s):root:	no nonequilibrium detected.
2021-08-19 13:28:27,464:(0.00s):root:	cataloging HybridTopologyFactories...
2021-08-19 13:28:27,466:(0.00s):root:		phase: solvent:
2021-08-19 13:28:27,467:(0.00s):root:		writing HybridTopologyFactory for phase solvent...
2021-08-19 13:28:27,469:(0.00s):relative:*** Generating RestCapablePMEHybridTopologyFactory ***
2021-08-19 13:28:27,535:(0.07s):relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'Mo

In [20]:
htf = setup_dict['hybrid_topology_factories']['solvent']

In [21]:
with open("cdk2_solvent.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no rest regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [22]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [23]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954754), ('HarmonicAngleForce', 29.19952360920909), ('PeriodicTorsionForce', 15.076747992617824), ('NonbondedForce', 1155.3185260687214), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954754), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [24]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [25]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899662), ('HarmonicAngleForce', 29.156055571071168), ('PeriodicTorsionForce', 15.080137722591825), ('NonbondedForce', 1167.4875392137392), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.519508183899662), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


### Test one alchemical region, one rest region

In [26]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle

In [27]:
# Create a htf
setup_directory = resource_filename("perses", "data/cdk2-example")

# Get options
from perses.app.setup_relative_calculation import getSetupOptions
yaml_filename = os.path.join(setup_directory, "cdk2_setup_repex.yaml")
setup_options = getSetupOptions(yaml_filename)

# Update options
for parameter in ['protein_pdb', 'ligand_file']:
    setup_options[parameter] = os.path.join(setup_directory, setup_options[parameter])
    
setup_options['rest_over_protocol'] = True
setup_options['phases'] = ['solvent']
setup_options['use_given_geometries'] = False
setup_options['rest_region'] = [44, 2197]
setup_options['validate_endstate_energies'] = False    

setup_dict = setup_relative_calculation.run_setup(setup_options, serialize_systems=False, build_samplers=False)

2021-08-19 13:28:35,793:(7.52s):root:	Detecting phases...
2021-08-19 13:28:35,795:(0.00s):root:		phases detected: ['complex', 'solvent', 'vacuum']
2021-08-19 13:28:35,797:(0.00s):root:No constraints will be removed
2021-08-19 13:28:35,799:(0.00s):root:No spectators
2021-08-19 13:28:35,800:(0.00s):root:			run_type is not specified; default to None
2021-08-19 13:28:35,802:(0.00s):root:	Detecting fe_type...
2021-08-19 13:28:35,804:(0.00s):root:		fe_type: repex
2021-08-19 13:28:35,806:(0.00s):root:			offline-freq not specified: default to 10.
2021-08-19 13:28:35,807:(0.00s):root:	'neglect_angles' detected: False.
2021-08-19 13:28:35,809:(0.00s):root:	'softcore_v2' not specified: default to 'False'
2021-08-19 13:28:35,811:(0.00s):root:	Creating 'output'...
2021-08-19 13:28:35,813:(0.00s):root:	detected ligand file: /home/zhangi/miniconda3/envs/perses-rest-over-protocol/lib/python3.8/site-packages/perses-0.9.1+35.gb0f7541-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf
2021-08-19 13:28:3

2021-08-19 13:28:37,919:(0.00s):geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
2021-08-19 13:28:37,920:(0.00s):geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
2021-08-19 13:28:37,922:(0.00s):geometry:		there are 2395 in the reference Nonbonded force
2021-08-19 13:28:37,980:(0.06s):geometry:Neglected angle terms : []
2021-08-19 13:28:37,982:(0.00s):geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
2021-08-19 13:28:37,983:(0.00s):geometry:extra torsions: {0: (2190, 2161, 2162, 2163, [1, Quantity(value=-0.019594642035228205, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 1]), 1: (2190, 2161, 2162, 2191, [1, Quantity(value=-3.1361896196768977, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 1]), 2: (2190, 2161, 2175, 2174, [1, Quantity(value=-0.008871224163264912, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole)

conducting subsequent work with the following platform: CUDA


2021-08-19 13:28:38,501:(0.52s):geometry:setting atoms_with_positions context new positions
2021-08-19 13:28:38,877:(0.38s):geometry:There are 1 new atoms


conducting subsequent work with the following platform: CUDA


...

2021-08-19 13:28:43,864:(1.01s):geometry:total reduced potential before atom placement: 48.41899607369697


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-08-19 13:28:45,034:(1.17s):geometry:total reduced energy added from growth system: -5.038066954807732
2021-08-19 13:28:45,036:(0.00s):geometry:final reduced energy 43.38092980084384
2021-08-19 13:28:45,038:(0.00s):geometry:sum of energies: 43.380929118889235
2021-08-19 13:28:45,040:(0.00s):geometry:magnitude of difference in the energies: 6.819546030101264e-07
2021-08-19 13:28:45,041:(0.00s):geometry:Final logp_proposal: 11.562670091155372
2021-08-19 13:28:45,181:(0.14s):root:	writing pickle output...


added energy components: [('CustomBondForce', 0.08798429542805747), ('CustomAngleForce', 0.04605409749261914), ('CustomTorsionForce', 0.005423562569706602), ('CustomBondForce', -5.177528910298116)]


2021-08-19 13:28:45,361:(0.18s):root:	successfully dumped pickle.
2021-08-19 13:28:45,367:(0.01s):root:	setup is complete.  Writing proposals and positions for each phase to top_prop dict...
2021-08-19 13:28:45,370:(0.00s):root:	writing atom_mapping.png
2021-08-19 13:28:45,426:(0.06s):root:	 steps per move application: 50
2021-08-19 13:28:45,429:(0.00s):root:	trajectory prefix: out
2021-08-19 13:28:45,430:(0.00s):root:	atom selection detected: not water
2021-08-19 13:28:45,432:(0.00s):root:	no nonequilibrium detected.
2021-08-19 13:28:45,433:(0.00s):root:	cataloging HybridTopologyFactories...
2021-08-19 13:28:45,435:(0.00s):root:		phase: solvent:
2021-08-19 13:28:45,436:(0.00s):root:		writing HybridTopologyFactory for phase solvent...
2021-08-19 13:28:45,438:(0.00s):relative:*** Generating RestCapablePMEHybridTopologyFactory ***
2021-08-19 13:28:45,507:(0.07s):relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'Mo

In [28]:
htf = setup_dict['hybrid_topology_factories']['solvent']

In [29]:
with open("cdk2_solvent_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [30]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/cdk2_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [31]:
test_bond_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954754), ('HarmonicAngleForce', 29.19952360920909), ('PeriodicTorsionForce', 15.076747992617824), ('NonbondedForce', 2263.9032169954517), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954754), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954754), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.563795580927208), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Scaling the bond force changes the energy


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [32]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/cdk2_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [33]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899662), ('HarmonicAngleForce', 30.50855620140993), ('PeriodicTorsionForce', 15.087836996961196), ('NonbondedForce', 2274.5789646221738), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.519508183899662), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


# 8mer in solvent

In [3]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle
from simtk import openmm, unit
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


In [4]:
# Test 8-mer peptide in solvent
peptide_filename = resource_filename('perses', 'data/8mer-example/4zuh_peptide_capped.pdb')
pdb = app.PDBFile(peptide_filename)
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
# barostat = None
modeller = app.Modeller(pdb.topology, pdb.positions)
 
system_generator = SystemGenerator(forcefields=forcefield_files,
                                   barostat=barostat,
                                   forcefield_kwargs={'removeCMMotion': False,
                                                      'ewaldErrorTolerance': 0.00025,
                                                      'constraints': app.HBonds,
                                                      'hydrogenMass': 4 * unit.amus},
                                   periodic_forcefield_kwargs=None,
                                   small_molecule_forcefield='gaff-2.11',
                                   nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff},
                                   molecules=None,
                                   cache=None)

modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
topology = modeller.getTopology()
positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(topology)



In [5]:
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.annihilation.relative import RepartitionedHybridTopologyFactory
from perses.tests.utils import validate_endstate_energies
import random

aminos = ['ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE','LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL']

ENERGY_THRESHOLD = 1e-6
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT

chain = 'C'
for res in topology.residues():
    if res.id == '2':
        wt_res = res.name
aminos_updated = [amino for amino in aminos if amino not in [wt_res, 'PRO', 'HIS', 'TRP', 'PHE', 'TYR']]
mutant = random.choice(aminos_updated)
# mutant = "MET"
print(f'Making mutation {wt_res}->{mutant}')

# Create point mutation engine to mutate residue at id 2 to random amino acid
point_mutation_engine = PointMutationEngine(wildtype_topology=topology,
                                            system_generator=system_generator,
                                            chain_id=chain,
                                            max_point_mutants=1,
                                            residues_allowed_to_mutate=['2'],  # the residue ids allowed to mutate
                                            allowed_mutations=[('2', mutant)],
                                            aggregate=True)  # always allow aggregation

# Create topology proposal
topology_proposal = point_mutation_engine.propose(current_system=system, current_topology=topology)

# Create geometry engine
from perses.rjmc.geometry import FFAllAngleGeometryEngine
geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                           use_sterics=False,
                                           n_bond_divisions=100,
                                           n_angle_divisions=180,
                                           n_torsion_divisions=360,
                                           verbose=True,
                                           storage=None,
                                           bond_softening_constant=1.0,
                                           angle_softening_constant=1.0,
                                           neglect_angles=False,
                                           use_14_nonbondeds=True)

# Create geometry proposal
new_positions, logp_proposal = geometry_engine.propose(topology_proposal, positions, beta,
                                                               validate_energy_bookkeeping=True)
logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, positions, beta,
                                            validate_energy_bookkeeping=True)

if not topology_proposal.unique_new_atoms:
    assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
    assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
    vacuum_added_valence_energy = 0.0
else:
    added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

if not topology_proposal.unique_old_atoms:
    assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
    assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
    subtracted_valence_energy = 0.0
else:
    subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


2021-08-19 13:37:50,182:(0.00s):proposal_generator:	Conducting polymer point mutation proposal...


Making mutation ALA->ARG


2021-08-19 13:37:50,468:(0.29s):atom_mapping:Molecules do not appear to share a common scaffold.
2021-08-19 13:37:50,470:(0.00s):atom_mapping:Proceeding with direct mapping of molecules, but please check atom mapping and the geometry of the ligands.
2021-08-19 13:37:50,491:(0.02s):proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
2021-08-19 13:37:50,493:(0.00s):proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD, atomic number=6), Atom(name=NE, atomic number=7), Atom(name=CZ, atomic number=6), Atom(name=NH1, atomic number=7), Atom(name=NH2, atomic number=7), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HG2, atomic number=1), Atom(name=HG3, atomic number=1), Atom(name=HD2, atomic number=1), Atom(name=HD3, atomic number=1), Atom(name=HE, atomic number=1), Atom(name=HH11, atomic number=1), Atom(name=HH12, atomic nu

conducting subsequent work with the following platform: CUDA


2021-08-19 13:37:53,057:(0.53s):geometry:setting atoms_with_positions context new positions


...

2021-08-19 13:38:00,160:(0.11s):geometry:	reduced angle potential = 2.233125127093818e-08.
2021-08-19 13:38:00,257:(0.10s):geometry:	reduced angle potential = 9.561051668702576e-06.
2021-08-19 13:38:00,365:(0.11s):geometry:	reduced angle potential = 9.265637866968462e-06.
2021-08-19 13:38:00,538:(0.17s):geometry:	beginning construction of no_nonbonded final system...
2021-08-19 13:38:00,540:(0.00s):geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
2021-08-19 13:38:00,798:(0.26s):geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
2021-08-19 13:38:00,801:(0.00s):geometry:	there are 61 bond forces in the no-nonbonded final system
2021-08-19 13:38:00,802:(0.00s):geometry:	there are 218 angle forces in the no-nonbonded final system
2021-08-19 13:38:00,804:(0.00s):geometry:	there are 399 tor

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-08-19 13:38:01,805:(1.00s):geometry:total reduced potential before atom placement: 168.36662826271765


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.04562463240715885), ('CustomAngleForce', 0.11963264577965821), ('CustomTorsionForce', 7.457414669869015), ('CustomBondForce', 11.49068603024672)]


2021-08-19 13:38:02,921:(1.12s):geometry:total reduced energy added from growth system: 19.11335797830255
2021-08-19 13:38:02,923:(0.00s):geometry:final reduced energy 187.47998570132236
2021-08-19 13:38:02,924:(0.00s):geometry:sum of energies: 187.4799862410202
2021-08-19 13:38:02,925:(0.00s):geometry:magnitude of difference in the energies: 5.396978401961405e-07
2021-08-19 13:38:02,926:(0.00s):geometry:Final logp_proposal: -30294.385345801762


In [6]:
flatten_exceptions = True
flatten_torsions = True
endstate = None
rest_region = None
htf = RestCapablePMEHybridTopologyFactory(topology_proposal=topology_proposal,
                     current_positions=atp.positions,
                     new_positions=new_positions,
                     rest_region=rest_region,
                     use_dispersion_correction=False,
                     generate_htf_for_testing=True,
                     interpolate_old_and_new_14s=flatten_exceptions
                    )

2021-08-19 13:38:03,033:(0.11s):relative:*** Generating RestCapablePMEHybridTopologyFactory ***
2021-08-19 13:38:03,132:(0.10s):relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-08-19 13:38:03,133:(0.00s):relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-08-19 13:38:03,135:(0.00s):relative:No unknown forces.
2021-08-19 13:38:03,136:(0.00s):relative:Nonbonded method to be used (i.e. from old system): 4
2021-08-19 13:38:03,137:(0.00s):relative:Adding and mapping old atoms to hybrid system...
2021-08-19 13:38:03,162:(0.03s):relative:Adding and mapping new atoms to hybrid system...
2021-08-19 13:38:03,164:(0.00s):relative:Added MonteCarloBarostat.
2021-08-19 13:38:03,165:(0.00s):relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=3.9005, y=0.0, z=0.0), unit=nanome

In [7]:
# Save htf
with open("8mer_solvent.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no rest region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [8]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [9]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.300105424153172), ('HarmonicAngleForce', 12.445685055931843), ('PeriodicTorsionForce', 156.2435090637808), ('NonbondedForce', -9977.178722665794), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.300105424153172), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [10]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)


In [11]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 8.593342580031933), ('HarmonicAngleForce', 392.8867372841539), ('PeriodicTorsionForce', 165.85183375283793), ('NonbondedForce', 4695.038642747033), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 8.593342580031933), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


### Test one alchemical region, one rest region

In [12]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle
from simtk import openmm, unit
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np

In [13]:
# Test 8-mer peptide in solvent
peptide_filename = resource_filename('perses', 'data/8mer-example/4zuh_peptide_capped.pdb')
pdb = app.PDBFile(peptide_filename)
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
# barostat = None
modeller = app.Modeller(pdb.topology, pdb.positions)
 
system_generator = SystemGenerator(forcefields=forcefield_files,
                                   barostat=barostat,
                                   forcefield_kwargs={'removeCMMotion': False,
                                                      'ewaldErrorTolerance': 0.00025,
                                                      'constraints': app.HBonds,
                                                      'hydrogenMass': 4 * unit.amus},
                                   periodic_forcefield_kwargs=None,
                                   small_molecule_forcefield='gaff-2.11',
                                   nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff},
                                   molecules=None,
                                   cache=None)

modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
topology = modeller.getTopology()
positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(topology)



In [14]:
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.annihilation.relative import RepartitionedHybridTopologyFactory
from perses.tests.utils import validate_endstate_energies
import random

aminos = ['ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE','LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL']

ENERGY_THRESHOLD = 1e-6
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT

chain = 'C'
for res in topology.residues():
    if res.id == '2':
        wt_res = res.name
aminos_updated = [amino for amino in aminos if amino not in [wt_res, 'PRO', 'HIS', 'TRP', 'PHE', 'TYR']]
mutant = random.choice(aminos_updated)
print(f'Making mutation {wt_res}->{mutant}')

# Create point mutation engine to mutate residue at id 2 to random amino acid
point_mutation_engine = PointMutationEngine(wildtype_topology=topology,
                                            system_generator=system_generator,
                                            chain_id=chain,
                                            max_point_mutants=1,
                                            residues_allowed_to_mutate=['2'],  # the residue ids allowed to mutate
                                            allowed_mutations=[('2', mutant)],
                                            aggregate=True)  # always allow aggregation

# Create topology proposal
topology_proposal = point_mutation_engine.propose(current_system=system, current_topology=topology)

# Create geometry engine
from perses.rjmc.geometry import FFAllAngleGeometryEngine
geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                           use_sterics=False,
                                           n_bond_divisions=100,
                                           n_angle_divisions=180,
                                           n_torsion_divisions=360,
                                           verbose=True,
                                           storage=None,
                                           bond_softening_constant=1.0,
                                           angle_softening_constant=1.0,
                                           neglect_angles=False,
                                           use_14_nonbondeds=True)

# Create geometry proposal
new_positions, logp_proposal = geometry_engine.propose(topology_proposal, positions, beta,
                                                               validate_energy_bookkeeping=True)
logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, positions, beta,
                                            validate_energy_bookkeeping=True)

if not topology_proposal.unique_new_atoms:
    assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
    assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
    vacuum_added_valence_energy = 0.0
else:
    added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

if not topology_proposal.unique_old_atoms:
    assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
    assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
    subtracted_valence_energy = 0.0
else:
    subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


2021-08-19 13:38:28,906:(25.10s):proposal_generator:	Conducting polymer point mutation proposal...
2021-08-19 13:38:29,070:(0.16s):atom_mapping:Molecules do not appear to share a common scaffold.
2021-08-19 13:38:29,072:(0.00s):atom_mapping:Proceeding with direct mapping of molecules, but please check atom mapping and the geometry of the ligands.
2021-08-19 13:38:29,087:(0.01s):proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
2021-08-19 13:38:29,091:(0.00s):proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD, atomic number=6), Atom(name=OE1, atomic number=8), Atom(name=OE2, atomic number=8), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HG2, atomic number=1), Atom(name=HG3, atomic number=1)]


Making mutation ALA->GLU


2021-08-19 13:38:30,431:(1.34s):geometry:propose: performing forward proposal
2021-08-19 13:38:30,441:(0.01s):geometry:propose: unique new atoms detected; proceeding to _logp_propose...
2021-08-19 13:38:30,443:(0.00s):geometry:Conducting forward proposal...
2021-08-19 13:38:30,444:(0.00s):geometry:Computing proposal order with NetworkX...
2021-08-19 13:38:30,452:(0.01s):geometry:number of atoms to be placed: 9
2021-08-19 13:38:30,459:(0.01s):geometry:Atom index proposal order is [21, 26, 29, 30, 31, 28, 24, 27, 25]
2021-08-19 13:38:30,460:(0.00s):geometry:omitted_bonds: []
2021-08-19 13:38:30,461:(0.00s):geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
2021-08-19 13:38:31,585:(1.12s):geometry:creating growth system...
2021-08-19 13:38:31,752:(0.17s):geometry:	creating bond force...
2021-08-19 13:38:31,754:(0.00s):geometry:	there are 65 bonds in reference force.
2021-08-19 13:38:31,756:(0.00s):geometry:	creating angle

conducting subsequent work with the following platform: CUDA


2021-08-19 13:38:35,536:(3.58s):geometry:setting atoms_with_positions context new positions


...

conducting subsequent work with the following platform: CUDA


2021-08-19 13:38:53,905:(1.04s):geometry:total reduced potential before atom placement: 168.36662826271765


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-08-19 13:38:55,151:(1.25s):geometry:total reduced energy added from growth system: 19.11335797830255
2021-08-19 13:38:55,154:(0.00s):geometry:final reduced energy 187.47998570132236
2021-08-19 13:38:55,156:(0.00s):geometry:sum of energies: 187.4799862410202
2021-08-19 13:38:55,157:(0.00s):geometry:magnitude of difference in the energies: 5.396978401961405e-07
2021-08-19 13:38:55,158:(0.00s):geometry:Final logp_proposal: -30393.957534332112


added energy components: [('CustomBondForce', 0.04562463240715885), ('CustomAngleForce', 0.11963264577965821), ('CustomTorsionForce', 7.457414669869015), ('CustomBondForce', 11.49068603024672)]


In [15]:
flatten_exceptions = True
flatten_torsions = True
endstate = None
rest_region = list(htf._atom_classes['unique_old_atoms']) + list(htf._atom_classes['unique_new_atoms'])
htf = RestCapablePMEHybridTopologyFactory(topology_proposal=topology_proposal,
                     current_positions=atp.positions,
                     new_positions=new_positions,
                     rest_region=rest_region,
                     use_dispersion_correction=False,
                     generate_htf_for_testing=True,
                     interpolate_old_and_new_14s=flatten_exceptions
                    )

2021-08-19 13:38:55,295:(0.14s):relative:*** Generating RestCapablePMEHybridTopologyFactory ***
2021-08-19 13:38:55,474:(0.18s):relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-08-19 13:38:55,477:(0.00s):relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-08-19 13:38:55,479:(0.00s):relative:No unknown forces.
2021-08-19 13:38:55,481:(0.00s):relative:Nonbonded method to be used (i.e. from old system): 4
2021-08-19 13:38:55,482:(0.00s):relative:Adding and mapping old atoms to hybrid system...
2021-08-19 13:38:55,520:(0.04s):relative:Adding and mapping new atoms to hybrid system...
2021-08-19 13:38:55,522:(0.00s):relative:Added MonteCarloBarostat.
2021-08-19 13:38:55,524:(0.00s):relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=3.9005, y=0.0, z=0.0), unit=nanome

In [16]:
# Save htf
with open("8mer_solvent_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [17]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [18]:
test_bond_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.300105424153172), ('HarmonicAngleForce', 12.445685055931843), ('PeriodicTorsionForce', 156.2435090637808), ('NonbondedForce', -10393.617741962595), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.300105424153172), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.300105424153172), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.277293107949593), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Scaling the bond force changes the energy


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [19]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [20]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 10.889625845390588), ('HarmonicAngleForce', 279.6437017041294), ('PeriodicTorsionForce', 169.90035224547114), ('NonbondedForce', 231019.52148371315), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 10.889625845390588), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


## Barnase:barstar (no naked charge fix)

### Test one alchemical region, no rest region

In [21]:
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from openeye import oechem

In [22]:
from __future__ import absolute_import

from perses.utils.openeye import createOEMolFromSDF, extractPositionsFromOEMol
from perses.annihilation.relative import HybridTopologyFactory, RepartitionedHybridTopologyFactory, RestCapablePMEHybridTopologyFactory
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine

import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import mdtraj as md
from openmmtools.constants import kB
from perses.tests.utils import validate_endstate_energies
from openff.toolkit.topology import Molecule
from openmmforcefields.generators import SystemGenerator

ENERGY_THRESHOLD = 1e-2
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
ring_amino_acids = ['TYR', 'PHE', 'TRP', 'PRO', 'HIS']

# Set up logger
import logging
_logger = logging.getLogger("setup")
_logger.setLevel(logging.INFO)

class PointMutationExecutor2(PointMutationExecutor):
    """
    Simple, stripped-down class to create a protein-ligand system and allow a mutation of a protein.
    this will allow support for the creation of _two_ relative free energy calculations:
        1. 'wildtype' - 'point mutant' complex hybrid.
        2. 'wildtype' - 'point mutant' protein hybrid (i.e. with ligand of interest unbound)
    Example (create full point mutation executor and run parallel tempering on both complex and apo phases):
        from pkg_resources import resource_filename
        protein_path = 'data/perses_jacs_systems/thrombin/Thrombin_protein.pdb'
        ligands_path = 'data/perses_jacs_systems/thrombin/Thrombin_ligands.sdf'
        protein_filename = resource_filename('openmmforcefields', protein_path)
        ligand_input = resource_filename('openmmforcefields', ligands_path)
        pm_delivery = PointMutationExecutor(protein_filename=protein_filename,
                                    mutation_chain_id='2',
                                    mutation_residue_id='198',
                                     proposed_residue='THR',
                                     phase='complex',
                                     conduct_endstate_validation=False,
                                     ligand_input=ligand_input,
                                     ligand_index=0,
                                     forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                                     barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                                     forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'nonbondedMethod': app.PME, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                                     small_molecule_forcefields='gaff-2.11')
        complex_htf = pm_delivery.get_complex_htf()
        apo_htf = pm_delivery.get_apo_htf()
        # Now we can build the hybrid repex samplers
        from perses.annihilation.lambda_protocol import LambdaProtocol
        from openmmtools.multistate import MultiStateReporter
        from perses.samplers.multistate import HybridRepexSampler
        from openmmtools import mcmc
        suffix = 'run'; selection = 'not water'; checkpoint_interval = 10; n_states = 11; n_cycles = 5000
        for htf in [complex_htf, apo_htf]:
            lambda_protocol = LambdaProtocol(functions='default')
            reporter_file = 'reporter.nc'
            reporter = MultiStateReporter(reporter_file, analysis_particle_indices = htf.hybrid_topology.select(selection), checkpoint_interval = checkpoint_interval)
            hss = HybridRepexSampler(mcmc_moves=mcmc.LangevinSplittingDynamicsMove(timestep= 4.0 * unit.femtoseconds,
                                                                                  collision_rate=5.0 / unit.picosecond,
                                                                                  n_steps=250,
                                                                                  reassign_velocities=False,
                                                                                  n_restart_attempts=20,
                                                                                  splitting="V R R R O R R R V",
                                                                                  constraint_tolerance=1e-06),
                                                                                  hybrid_factory=htf, online_analysis_interval=10)
            hss.setup(n_states=n_states, temperature=300*unit.kelvin, storage_file=reporter, lambda_protocol=lambda_protocol, endstates=False)
            hss.extend(n_cycles)
    """
    def __init__(self,
                 protein_filename,
                 mutation_chain_id,
                 mutation_residue_id,
                 proposed_residue,
                 phase='complex',
                 conduct_endstate_validation=True,
                 ligand_input=None,
                 ligand_index=0,
                 allow_undefined_stereo_sdf=False,
                 water_model='tip3p',
                 ionic_strength=0.15 * unit.molar,
                 forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                 barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                 forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 0.00025, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                 periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
                 nonperiodic_forcefield_kwargs=None,
                 small_molecule_forcefields='gaff-2.11',
                 complex_box_dimensions=None,
                 apo_box_dimensions=None,
                 flatten_torsions=False,
                 flatten_exceptions=False,
                 generate_unmodified_hybrid_topology_factory=True,
                 generate_rest_capable_hybrid_topology_factory=False,
                 generate_rest_capable_pme_hybrid_topology_factory=False,
                 rest_region=None,
                 **kwargs):
        """
        arguments
            protein_filename : str
                path to protein (to mutate); .pdb
            mutation_chain_id : str
                name of the chain to be mutated
            mutation_residue_id : str
                residue id to change
            proposed_residue : str
                three letter code of the residue to mutate to
            phase : str, default complex
                if phase == vacuum, then the complex will not be solvated with water; else, it will be solvated with tip3p
            conduct_endstate_validation : bool, default True
                whether to conduct an endstate validation of the HybridTopologyFactory. If using the RepartitionedHybridTopologyFactory,
                endstate validation cannot and will not be conducted.
            ligand_input : str, default None
                path to ligand of interest (i.e. small molecule or protein); .sdf or .pdb
            ligand_index : int, default 0
                which ligand to use
            allow_undefined_stereo_sdf : bool, default False
                whether to allow an SDF file to contain undefined stereocenters
            water_model : str, default 'tip3p'
                solvent model to use for solvation
            ionic_strength : float * unit.molar, default 0.15 * unit.molar
                the total concentration of ions (both positive and negative) to add using Modeller.
                This does not include ions that are added to neutralize the system.
                Note that only monovalent ions are currently supported.
            forcefield_files : list of str, default ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
                forcefield files for proteins and solvent
            barostat : openmm.MonteCarloBarostat, default openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
                barostat to use
            forcefield_kwargs : dict, default {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus}
                forcefield kwargs for system parametrization
            periodic_forcefield_kwargs : dict, default {'nonbondedMethod': app.PME}
                periodic forcefield kwargs for system parametrization
            nonperiodic_forcefield_kwargs : dict, default None
                non-periodic forcefield kwargs for system parametrization
            small_molecule_forcefields : str, default 'gaff-2.11'
                the forcefield string for small molecule parametrization
            complex_box_dimensions : Vec3, default None
                define box dimensions of complex phase;
                if None, padding is 1nm
            apo_box_dimensions :  Vec3, default None
                define box dimensions of apo phase phase;
                if None, padding is 1nm
            flatten_torsions : bool, default False
                in the htf, flatten torsions involving unique new atoms at lambda = 0 and unique old atoms are lambda = 1
            flatten_exceptions : bool, default False
                in the htf, flatten exceptions involving unique new atoms at lambda = 0 and unique old atoms at lambda = 1
            generate_unmodified_hybrid_topology_factory : bool, default True
                whether to generate a vanilla HybridTopologyFactory
            generate_rest_capable_hybrid_topology_factory : bool, default False
                whether to generate a RepartitionedHybridTopologyFactory
        TODO : allow argument for spectator ligands besides the 'ligand_file'
        """
        from openeye import oechem

        # First thing to do is load the apo protein to mutate...
        protein_pdbfile = open(protein_filename, 'r')
        protein_pdb = app.PDBFile(protein_pdbfile)
        protein_pdbfile.close()
        protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
        protein_topology = protein_md_topology.to_openmm()
        protein_n_atoms = protein_md_topology.n_atoms

        # Load the ligand, if present
        molecules = []
        if ligand_input:
            if isinstance(ligand_input, str):
                if ligand_input.endswith('.sdf'): # small molecule
                        ligand_mol = createOEMolFromSDF(ligand_file, index=ligand_index, allow_undefined_stereo=allow_undefined_stereo_sdf)
                        molecules.append(Molecule.from_openeye(ligand_mol, allow_undefined_stereo=False))
                        ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_mol),  forcefield_generators.generateTopologyFromOEMol(ligand_mol)
                        ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                        ligand_n_atoms = ligand_md_topology.n_atoms

                if ligand_input.endswith('pdb'): # protein
                    ligand_pdbfile = open(ligand_input, 'r')
                    ligand_pdb = app.PDBFile(ligand_pdbfile)
                    ligand_pdbfile.close()
                    ligand_positions, ligand_topology, ligand_md_topology = ligand_pdb.positions, ligand_pdb.topology, md.Topology.from_openmm(
                        ligand_pdb.topology)
                    ligand_n_atoms = ligand_md_topology.n_atoms

            elif isinstance(ligand_input, oechem.OEMol): # oemol object
                molecules.append(Molecule.from_openeye(ligand_input, allow_undefined_stereo=False))
                ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_input),  forcefield_generators.generateTopologyFromOEMol(ligand_input)
                ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                ligand_n_atoms = ligand_md_topology.n_atoms

            else:
                _logger.warning(f'ligand filetype not recognised. Please provide a path to a .pdb or .sdf file')
                return

            # Now create a complex
            complex_md_topology = protein_md_topology.join(ligand_md_topology)
            complex_topology = complex_md_topology.to_openmm()
            complex_positions = unit.Quantity(np.zeros([protein_n_atoms + ligand_n_atoms, 3]), unit=unit.nanometers)
            complex_positions[:protein_n_atoms, :] = protein_positions
            complex_positions[protein_n_atoms:, :] = ligand_positions

        # Now for a system_generator
        self.system_generator = SystemGenerator(forcefields=forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                small_molecule_forcefield=small_molecule_forcefields,
                                                molecules=molecules,
                                                cache=None)

        # Solvate apo and complex...
        apo_input = list(self._solvate(protein_topology, protein_positions, water_model, phase, ionic_strength, apo_box_dimensions))
        inputs = [apo_input]
        if ligand_input:
            inputs.append(self._solvate(complex_topology, complex_positions, water_model, phase, ionic_strength, complex_box_dimensions))

        geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                                use_sterics=False,
                                                n_bond_divisions=100,
                                                n_angle_divisions=180,
                                                n_torsion_divisions=360,
                                                verbose=True,
                                                storage=None,
                                                bond_softening_constant=1.0,
                                                angle_softening_constant=1.0,
                                                neglect_angles = False,
                                                use_14_nonbondeds = True)


        # Run pipeline...
        htfs = []
        for is_complex, (top, pos, sys) in enumerate(inputs):
            point_mutation_engine = PointMutationEngine(wildtype_topology=top,
                                                                 system_generator=self.system_generator,
                                                                 chain_id=mutation_chain_id, # Denote the chain id allowed to mutate (it's always a string variable)
                                                                 max_point_mutants=1,
                                                                 residues_allowed_to_mutate=[mutation_residue_id], # The residue ids allowed to mutate
                                                                 allowed_mutations=[(mutation_residue_id, proposed_residue)], # The residue ids allowed to mutate with the three-letter code allowed to change
                                                                 aggregate=True) # Always allow aggregation

            topology_proposal = point_mutation_engine.propose(sys, top)

            # Only validate energy bookkeeping if the WT and proposed residues do not involve rings
            old_res = [res for res in top.residues() if res.id == mutation_residue_id][0]
            validate_bool = False if old_res.name in ring_amino_acids or proposed_residue in ring_amino_acids else True
            new_positions, logp_proposal = geometry_engine.propose(topology_proposal, pos, beta,
                                                                   validate_energy_bookkeeping=validate_bool)

            #check for charge change...
            charge_diff = point_mutation_engine._get_charge_difference(current_resname=topology_proposal.old_topology.residue_topology.name,
                                                                       new_resname=topology_proposal.new_topology.residue_topology.name)
            _logger.info(f"charge diff: {charge_diff}")
            if charge_diff != 0:
                new_water_indices_to_ionize = point_mutation_engine.get_water_indices(charge_diff, new_positions, topology_proposal.new_topology, radius=0.8)
                _logger.info(f"new water indices to ionize {new_water_indices_to_ionize}")
                self._get_ion_and_water_parameters(topology_proposal.old_system, topology_proposal.old_topology)
                self._transform_waters_into_ions(new_water_indices_to_ionize, topology_proposal.new_system, charge_diff)
                PointMutationExecutor._modify_atom_classes(new_water_indices_to_ionize, topology_proposal)


            logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, pos, beta,
                                                        validate_energy_bookkeeping=validate_bool)
            if generate_unmodified_hybrid_topology_factory:
                repartitioned_endstate = None
                self.generate_htf(HybridTopologyFactory, topology_proposal, pos, new_positions, flatten_exceptions, flatten_torsions, repartitioned_endstate, is_complex, rest_region)
            if generate_rest_capable_hybrid_topology_factory:
                 for repartitioned_endstate in [0, 1]:
                    self.generate_htf(RepartitionedHybridTopologyFactory, topology_proposal, pos, new_positions, flatten_exceptions, flatten_torsions, repartitioned_endstate, is_complex, rest_region)
            if generate_rest_capable_pme_hybrid_topology_factory:
                repartitioned_endstate = None
                self.generate_htf(RestCapablePMEHybridTopologyFactory, topology_proposal, pos, new_positions, flatten_exceptions, flatten_torsions, repartitioned_endstate, is_complex, rest_region)

            if not topology_proposal.unique_new_atoms:
                assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
                assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
            else:
                added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

            if not topology_proposal.unique_old_atoms:
                assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
                assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
                subtracted_valence_energy = 0.0
            else:
                subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


            if conduct_endstate_validation and repartitioned_endstate is None:
                zero_state_error, one_state_error = validate_endstate_energies(forward_htf._topology_proposal, forward_htf, added_valence_energy, subtracted_valence_energy, beta=beta, ENERGY_THRESHOLD=ENERGY_THRESHOLD)
                if zero_state_error > ENERGY_THRESHOLD:
                    _logger.warning(f"Reduced potential difference of the nonalchemical and alchemical Lambda = 0 state is above the threshold ({ENERGY_THRESHOLD}): {zero_state_error}")
                if one_state_error > ENERGY_THRESHOLD:
                    _logger.warning(f"Reduced potential difference of the nonalchemical and alchemical Lambda = 1 state is above the threshold ({ENERGY_THRESHOLD}): {one_state_error}")
            else:
                pass
            
    def generate_htf(self, factory, topology_proposal, old_positions, new_positions, flatten_exceptions, flatten_torsions, repartitioned_endstate, is_complex, rest_region):
        htf = factory(topology_proposal=topology_proposal,
                                      current_positions=old_positions,
                                      new_positions=new_positions,
                                      use_dispersion_correction=False,
                                      functions=None,
                                      softcore_alpha=None,
                                      bond_softening_constant=1.0,
                                      angle_softening_constant=1.0,
                                      soften_only_new=False,
                                      neglected_new_angle_terms=[],
                                      neglected_old_angle_terms=[],
                                      softcore_LJ_v2=True,
                                      softcore_electrostatics=True,
                                      softcore_LJ_v2_alpha=0.85,
                                      softcore_electrostatics_alpha=0.3,
                                      softcore_sigma_Q=1.0,
                                      interpolate_old_and_new_14s=flatten_exceptions,
                                      omitted_terms=None,
                                      endstate=repartitioned_endstate,
                                      flatten_torsions=flatten_torsions,
                                      rest_region=rest_region)
        if is_complex:
            if factory == HybridTopologyFactory or RestCapablePMEHybridTopologyFactory:
                self.complex_htf = htf
            elif factory == RepartitionedHybridTopologyFactory:
                if repartitioned_endstate == 0:
                    self.complex_rhtf_0 = htf
                elif repartitioned_endstate == 1:
                    self.complex_rhtf_1 = htf
        else:
            if factory == HybridTopologyFactory or RestCapablePMEHybridTopologyFactory:
                self.apo_htf = htf
            elif factory == RepartitionedHybridTopologyFactory:
                if repartitioned_endstate == 0:
                    self.apo_rhtf_0 = htf
                elif repartitioned_endstate == 1:
                    self.apo_rhtf_1 = htf

    

In [23]:
solvent_delivery = PointMutationExecutor2("../../input/1brs_barstar_renumbered.pdb",
                        '1',
                        "42",
                        "ALA",
                        ligand_input="../../input/1brs_barnase_renumbered.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True, 
                        conduct_endstate_validation=False,
                        generate_unmodified_hybrid_topology_factory=False,
                        generate_rest_capable_hybrid_topology_factory=False,
                        generate_rest_capable_pme_hybrid_topology_factory=True
                       )

2021-08-19 13:39:25,036:(28.86s):setup:solvating at 0.05 M using tip3p
2021-08-19 13:39:32,965:(7.93s):setup:solvating at 0.05 M using tip3p
2021-08-19 13:39:49,205:(16.24s):proposal_generator:	Conducting polymer point mutation proposal...
2021-08-19 13:39:49,857:(0.65s):atom_mapping:Molecules do not appear to share a common scaffold.
2021-08-19 13:39:49,859:(0.00s):atom_mapping:Proceeding with direct mapping of molecules, but please check atom mapping and the geometry of the ligands.
2021-08-19 13:39:49,868:(0.01s):proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG2, atomic number=6), Atom(name=OG1, atomic number=8), Atom(name=HB, atomic number=1), Atom(name=HG1, atomic number=1), Atom(name=HG21, atomic number=1), Atom(name=HG22, atomic number=1), Atom(name=HG23, atomic number=1)]
2021-08-19 13:39:49,869:(0.00s):proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
2021-08-19

conducting subsequent work with the following platform: CUDA


2021-08-19 13:40:01,907:(2.26s):geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


2021-08-19 13:40:02,907:(1.00s):geometry:There are 4 new atoms
2021-08-19 13:40:02,913:(0.01s):geometry:	reduced angle potential = 0.2280726529720325.
2021-08-19 13:40:03,122:(0.21s):geometry:	reduced angle potential = 1.507208055442705.
2021-08-19 13:40:03,327:(0.20s):geometry:	reduced angle potential = 0.7780027623195036.
2021-08-19 13:40:03,533:(0.21s):geometry:	reduced angle potential = 0.06919178624831976.
2021-08-19 13:40:04,039:(0.51s):geometry:	beginning construction of no_nonbonded final system...
2021-08-19 13:40:04,043:(0.00s):geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
2021-08-19 13:40:05,047:(1.00s):geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
2021-08-19 13:40:05,049:(0.00s):geometry:	there are 733 bond forces in the no-nonbonded final system
2021-08-19 13:40:0

...

2021-08-19 13:41:52,338:(4.17s):geometry:total reduced potential before atom placement: 5668.103368699857


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-08-19 13:41:54,840:(2.50s):geometry:total reduced energy added from growth system: -60.14549627276338
2021-08-19 13:41:54,843:(0.00s):geometry:final reduced energy 5607.957872303956
2021-08-19 13:41:54,845:(0.00s):geometry:sum of energies: 5607.957872427094
2021-08-19 13:41:54,847:(0.00s):geometry:magnitude of difference in the energies: 1.231379371802177e-07
2021-08-19 13:41:54,848:(0.00s):geometry:Final logp_proposal: -8985.459730894416
2021-08-19 13:41:54,977:(0.13s):relative:*** Generating RestCapablePMEHybridTopologyFactory ***


added energy components: [('CustomBondForce', 0.6231737166566027), ('CustomAngleForce', 2.1618390579023568), ('CustomTorsionForce', 18.205977349832974), ('CustomBondForce', -81.13648639715531)]


2021-08-19 13:41:56,359:(1.38s):relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-08-19 13:41:56,363:(0.00s):relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-08-19 13:41:56,365:(0.00s):relative:No unknown forces.
2021-08-19 13:41:56,367:(0.00s):relative:Nonbonded method to be used (i.e. from old system): 4
2021-08-19 13:41:56,369:(0.00s):relative:Adding and mapping old atoms to hybrid system...
2021-08-19 13:41:56,578:(0.21s):relative:Adding and mapping new atoms to hybrid system...
2021-08-19 13:41:56,580:(0.00s):relative:Added MonteCarloBarostat.
2021-08-19 13:41:56,581:(0.00s):relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=7.062600000000001, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=7.062600000000001, z=0.0), unit=nanometer), Quanti

In [24]:
pickle.dump(solvent_delivery.get_apo_htf(), open("T42A_apo.pickle", "wb" ))
pickle.dump(solvent_delivery.get_complex_htf(), open("T42A_complex.pickle", "wb" ))


#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [25]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [26]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 569.5926176486894), ('HarmonicAngleForce', 1323.9862513751366), ('PeriodicTorsionForce', 3795.5154898495653), ('NonbondedForce', 57388803.7320065), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 569.5926176486894), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [27]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [28]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 568.9727472639595), ('HarmonicAngleForce', 1476.9144856552205), ('PeriodicTorsionForce', 3787.2316080556297), ('NonbondedForce', 57389057.42211673), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 568.9727472639595), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


In [29]:
rest_region = list(htf._atom_classes['unique_old_atoms']) + list(htf._atom_classes['unique_new_atoms'])

### Test one alchemical region, one rest region

In [30]:
solvent_delivery = PointMutationExecutor2("../../input/1brs_barstar_renumbered.pdb",
                        '1',
                        "42",
                        "ALA",
                        ligand_input="../../input/1brs_barnase_renumbered.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True, 
                        conduct_endstate_validation=False,
                        generate_unmodified_hybrid_topology_factory=False,
                        generate_rest_capable_hybrid_topology_factory=False,
                        generate_rest_capable_pme_hybrid_topology_factory=True,
                        rest_region=rest_region
                       )

2021-08-19 13:44:44,913:(164.07s):setup:solvating at 0.05 M using tip3p
2021-08-19 13:44:48,187:(3.27s):setup:solvating at 0.05 M using tip3p
2021-08-19 13:44:57,223:(9.04s):proposal_generator:	Conducting polymer point mutation proposal...
2021-08-19 13:44:57,581:(0.36s):atom_mapping:Molecules do not appear to share a common scaffold.
2021-08-19 13:44:57,584:(0.00s):atom_mapping:Proceeding with direct mapping of molecules, but please check atom mapping and the geometry of the ligands.
2021-08-19 13:44:57,598:(0.01s):proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG2, atomic number=6), Atom(name=OG1, atomic number=8), Atom(name=HB, atomic number=1), Atom(name=HG1, atomic number=1), Atom(name=HG21, atomic number=1), Atom(name=HG22, atomic number=1), Atom(name=HG23, atomic number=1)]
2021-08-19 13:44:57,599:(0.00s):proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
2021-08-19

conducting subsequent work with the following platform: CUDA


2021-08-19 13:45:08,827:(0.79s):geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


2021-08-19 13:45:09,590:(0.76s):geometry:There are 4 new atoms
2021-08-19 13:45:09,596:(0.01s):geometry:	reduced angle potential = 0.11876324593478398.
2021-08-19 13:45:09,814:(0.22s):geometry:	reduced angle potential = 0.7510667581037692.
2021-08-19 13:45:10,014:(0.20s):geometry:	reduced angle potential = 0.2581301719214334.
2021-08-19 13:45:10,203:(0.19s):geometry:	reduced angle potential = 0.15070124282156389.
2021-08-19 13:45:10,737:(0.53s):geometry:	beginning construction of no_nonbonded final system...
2021-08-19 13:45:10,740:(0.00s):geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
2021-08-19 13:45:11,821:(1.08s):geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
2021-08-19 13:45:11,826:(0.00s):geometry:	there are 733 bond forces in the no-nonbonded final system
2021-08-19 13:45

...

2021-08-19 13:48:19,816:(2.77s):geometry:total reduced potential before atom placement: 5668.103368699857


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-08-19 13:48:21,634:(1.82s):geometry:total reduced energy added from growth system: -60.14549627276338
2021-08-19 13:48:21,638:(0.00s):geometry:final reduced energy 5607.957872303956
2021-08-19 13:48:21,639:(0.00s):geometry:sum of energies: 5607.957872427094
2021-08-19 13:48:21,641:(0.00s):geometry:magnitude of difference in the energies: 1.231379371802177e-07
2021-08-19 13:48:21,643:(0.00s):geometry:Final logp_proposal: -9001.549465314636


added energy components: [('CustomBondForce', 0.6231737166566027), ('CustomAngleForce', 2.1618390579023568), ('CustomTorsionForce', 18.205977349832974), ('CustomBondForce', -81.13648639715531)]


2021-08-19 13:48:21,798:(0.16s):relative:*** Generating RestCapablePMEHybridTopologyFactory ***
2021-08-19 13:48:23,147:(1.35s):relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-08-19 13:48:23,151:(0.00s):relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-08-19 13:48:23,154:(0.00s):relative:No unknown forces.
2021-08-19 13:48:23,157:(0.00s):relative:Nonbonded method to be used (i.e. from old system): 4
2021-08-19 13:48:23,160:(0.00s):relative:Adding and mapping old atoms to hybrid system...
2021-08-19 13:48:23,404:(0.24s):relative:Adding and mapping new atoms to hybrid system...
2021-08-19 13:48:23,406:(0.00s):relative:Added MonteCarloBarostat.
2021-08-19 13:48:23,407:(0.00s):relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=7.062600000000001, y=0.0, z=0.0), 

In [31]:
pickle.dump(solvent_delivery.get_apo_htf(), open("T42A_apo_scale_region.pickle", "wb" ))
pickle.dump(solvent_delivery.get_complex_htf(), open("T42A_complex_scale_region.pickle", "wb" ))


#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [32]:
# Load htf
with open("T42A_complex_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [33]:
test_bond_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 569.5926176486894), ('HarmonicAngleForce', 1323.9862513751366), ('PeriodicTorsionForce', 3795.5154898495653), ('NonbondedForce', 57389164.06569195), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 569.5926176486894), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 569.5926176486894), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 569.1975119857644), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Scaling the bond force changes the energy


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [34]:
# Load htf
with open("T42A_complex_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [35]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 569.160370778044), ('HarmonicAngleForce', 1471.3540641425589), ('PeriodicTorsionForce', 3787.0026523129745), ('NonbondedForce', 57389418.255840376), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 569.160370778044), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


In [ ]:
# TO DO: actually test the scaling of the energies matches